In [ ]:
import pandas as pd
from openai import OpenAI
import time
import re

In [ ]:
base_url = ""
api_key = ""
client = OpenAI(api_key=api_key, base_url=base_url)

In [ ]:
# prompt_basic = f'''/* 指令 */
# 你是一位专业的中文语言分析专家。你会接收到一个中文谐音词作为输入内容，请准确将其还原为原本的词汇，然后只输出一个符合 {"原词": "XXX"} 格式的 JSON 数据，这里的 “XXX” 就是你所输入的谐音词所对应的原本词汇。

# /* 输入 */
# 当前的输入是：'''

In [ ]:
# prompt_context = f'''/* 指令 */
# 你是一位专业的中文语言分析专家。你会收到含有一个谐音词的中文句子和该句子中的中文谐音词作为输入内容，请准确将其中的谐音词还原为原本的词汇，直接输出且只输出一个符合 {{"原词": "XXX"}} 格式的 JSON 数据，这里的 “XXX” 就是输入谐音词所对应的原本的中文词汇。输出中只能含有该json数据，而不能包含其他任何多余信息。


# /* 输入 */
# 句子输入：
# 谐音词：'''

In [ ]:
# prompt_pinyin = f'''/* 指令 */
# 你是一位专业的中文语言分析专家。你会收到含有一个谐音词的中文句子和该句子中的拼音，以及句子中的谐音词和谐音词的拼音作为输入内容，请准确将其中的谐音词还原为原本的词汇，直接输出且只输出一个符合 {{"原词": "XXX"}} 格式的 JSON 数据，这里的 “XXX” 就是输入谐音词所对应的原本的中文词汇。输出中只能含有该json数据，而不能包含其他任何多余信息。

# /* 输入 */
# 句子输入：
# 句子输入的拼音：
# 谐音词：
# 谐音词的拼音：'''

In [ ]:
# prompt_cot = f'''/* 指令 */
# 你是一位专业的中文语言分析专家。你会收到含有一个谐音词的中文句子和该句子中的谐音词作为输入内容，请首先给出思考推理的过程，然后准确地将句子的谐音词还原为原本的词汇，最后只输出样式为 {"推理过程": "XXX","原词": "XXX"} 的 JSON 数据，这里的第一个 "XXX" 是你推理的过程，第二个 "XXX" 就是你所输入的谐音词所对应的原本词汇。 输出中只能含有该json样式的数据，而不能包含其他任何多余信息。

# /* 输入 */
# 句子输入：
# 谐音词：'''

In [ ]:
# prompt_mot = f'''/* 指令 */
# 你是一位专业的中文语言分析专家。你会收到含有一个谐音词的中文句子和该句子中的谐音词作为输入内容，请首先给出思考推理的过程，思考过程的样式仅参考我给出的示例，然后准确地将句子的谐音词还原为原本的词汇，最后只输出样式为 {"推理过程": "XXX","原词": "XXX"} 的 JSON 数据，这里的第一个 "XXX" 是你推理的过程，第二个 "XXX" 就是你所输入的谐音词所对应的原本词汇。 输出中只能含有该json样式的数据，而不能包含其他任何多余信息。

# /* 示例 */
# 句子输入：不要对我人参公鸡，否则我让管理员过来处理了。
# 谐音词:人参公鸡
# 输出为：{"推理过程":"'人参公鸡'的拼音是[[ren2],[shen1],[gong1],[ji1]]，原词应该为'人身攻击'，拼音是 [[ren2],[shen1],[gong1],[ji1]]。这是属于完全的同音字置换形成的谐音词现象，拼音拼写（发声位置）以及音调没有发生任何变化。这个谐音词中的'参'，'公'和'鸡'字属于遭到置换的字。他们分别经历将'身'替换为'参'，将'攻'替换为'公'，将'击'替换为'鸡'。这一现象仅改变了汉字写法，保持发音一致形成了谐音效果","原词"："人身攻击"}
# 句子输入：你肿么了？身体不舒服吗？
# 谐音词:肿么了
# 输出为：{{"推理过程":"'肿么了'的拼音[[zhong3],[me5],[le5]]，原词应该为'怎么了'，拼音为[[zen3],[me5],[le5]]。这是属于单纯的拼音拼写发生替换（发声位置发生变化）但是音调没有发生变化从而形成的谐音词现象。这个谐音词中的'肿'属于遭到置换的字。它的拼音拼写中的zh变成了z， IPA音标从[tʂ]变为[ts]，发音位置从卷舌擦音变为齿龈塞擦音，而且'怎'替换为'肿'。这一现象不仅改变了汉字写法而且改变了发音位置从而形成了谐音效果","原词":"怎么了"}
# 句子输入：这个综艺请的所谓专家根本是个叫兽！
# 谐音词：叫兽
# 输出为：{{"推理过程":"'叫兽'的拼音[[jiao4],[shou4]],原词应该为'教授',拼音为[[jiao4],[shou4]]。这是属于完全的同音字置换形成的谐音词现象，拼音拼写（发声位置）以及音调没有发生任何变化。这个谐音词中'叫'和'兽'字属于遭到替换的字。他们分别经历将"教"替换为'叫'，将"授"替换为"兽。这一现象仅改变了汉字写法，保持发音一致形成了谐音效果","原词"："教授"}}"

# /* 输入 */
# 句子输入：
# 谐音词：'''

In [ ]:
# prompt_few_shot = f'''/* 指令 */
# 你是一位专业的中文语言分析专家。你会收到含有一个谐音词的中文句子和该句子中的中文谐音词作为输入内容，请准确将其中的谐音词还原为原本的词汇，直接输出且只输出一个符合 {{"原词": "XXX"}} 格式的 JSON 数据，这里的 “XXX” 就是输入谐音词所对应的原本的中文词汇。输出中只能含有该json数据，而不能包含其他任何多余信息。

# /* 示例 */
# （1）句子输入：不要对我人参公鸡，否则我让管理员过来处理了。
# 谐音词:人参公鸡
# 输出为：{{"原词": "人身攻击"}}
# （2）句子输入：你肿么了？身体不舒服吗？
# 谐音词:肿么了
# 输出为：{{"原词": "怎么了"}}
# （3）句子输入：这个综艺请的所谓专家根本是个叫兽！
# 谐音词:叫兽
# 输出为：{{"原词":"教授"}}

# /* 输入 */
# 句子输入：
# 谐音词：'''

#basic prompt

In [ ]:
input_df = pd.read_csv('variant1.csv')
input_terms = input_df.iloc[:, 2].tolist()
print(input_terms)

In [ ]:
from tqdm import tqdm

output_data = []

for term in tqdm(input_terms, desc="生成进度"):
    prompt = f'''\
/* 指令 */
你是一位专业的中文语言分析专家。你会接收到一个中文谐音词作为输入内容，请准确将其还原为原本的词汇，然后只输出一个符合 {{"原词": "XXX"}} 格式的 JSON 数据，这里的 “XXX” 就是你所输入的谐音词所对应的原本词汇。

/* 输入 */
当前的输入是：
{term}'''# we should replace the prompt from previous code blocks


    response = client.chat.completions.create(
        model="deepseek-r1",
        messages=[
            {"role": "system", "content": "你是一位专业的中文语言分析专家。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        stream=False
    )

    raw_output = response.choices[0].message.content
    clean_output = re.sub(r'\s*<think>.*?</think>\s*', '', raw_output, flags=re.DOTALL)

    output_data.append({
        "word": term,
        "sentence": clean_output
    })

    print(f"已完成: {term} -> {clean_output}")

# 保存结果
pd.DataFrame(output_data).to_csv('output.csv', index=False, encoding='utf-8-sig')